### Install PyMySQL

In [2]:
!pip install pymysql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 262 kB/s 


In [3]:
!pip install python-dotenv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

True

### Import PyMySQL

In [5]:
import pymysql

### MySQL Configuration

In [6]:
class config:
  HOST = os.getenv('MYSQL_HOST')
  PORT = int(os.getenv('MYSQL_PORT'))
  USER = os.getenv('MYSQL_USER')
  PASSWORD = os.getenv('MYSQL_PASSWORD')
  DB = os.getenv('MYSQL_DB')
  CHARSET = os.getenv('MYSQL_CHARSET')


### Connect DB

In [7]:
connection = pymysql.connect(host=config.HOST,
                             port=config.PORT,
                             user=config.USER,
                             password=config.PASSWORD,
                             db=config.DB,
                             charset=config.CHARSET)

### Show tables in DB

In [8]:
with connection.cursor() as cursor:
  cursor.execute("show tables;")
  tables = cursor.fetchall()
tables


(('audible_data',), ('audible_transaction',))

In [9]:
import pandas as pd

In [10]:
sql = 'SELECT * FROM audible_data '
audible_data = pd.read_sql(sql,connection)
audible_data = audible_data.set_index('Book_ID')
audible_data

,Book Title,Book Subtitle,Book Author,Book Narrator,Audio Runtime,Audiobook_Type,Categories,Rating,Total No. of Ratings,Price
Book_ID,,,,,,,,,,
1,Bamboozled by Jesus,How God Tricked Me into the Life of My Dreams,Yvonne Orji,Yvonne Orji,6 hrs and 31 mins,Unabridged Audiobook,Biographies & Memoirs,5,47.0,$29.65
2,Sixth Realm Part 1,"A LitRPG Fantasy Series (The Ten Realms, Book 6)",Michael Chatfield,Neil Hellegers,13 hrs and 33 mins,Unabridged Audiobook,Science Fiction & Fantasy,4.5,98.0,$24.95
3,Go Tell the Bees That I Am Gone,"Outlander, Book 9",Diana Gabaldon,Davina Porter,27 hrs and 30 mins,Unabridged Audiobook,Science Fiction & Fantasy,None,NaN,$41.99
4,How the Word Is Passed,A Reckoning with the History of Slavery Across...,Clint Smith,Clint Smith,10 hrs and 7 mins,Unabridged Audiobook,History,None,NaN,$29.65
5,The Devil May Dance,A Novel,Jake Tapper,Rob Shapiro,10 hrs and 12 mins,Unabridged Audiobook,"Mystery, Thriller & Suspense",4.5,108.0,$29.65
...,...,...,...,...,...,...,...,...,...,...
2265,Kama Sutra,Become a Master of the Art of Love Making. Tan...,Eva Becker,ChasMandala,3 hrs and 29 mins,Unabridged Audiobook,Erotica,5,203.0,$14.95
2266,Meditación Práctica para Principiantes [Prac...,Una guía para principiantes para aliviar la an...,Taylor Knox,William Ditilio,3 hrs and 14 mins,Unabridged Audiobook,Health & Wellness,5,203.0,$14.95
2267,Forgotten,"The Untold Story of D-day's Black Heroes, at H...",Linda Hervieux,Ron Butler,9 hrs and 33 mins,Unabridged Audiobook,History,4.6,70.0,$24.95


In [12]:
sql = 'SELECT * FROM audible_transaction '
audible_transaction = pd.read_sql(sql,connection)
audible_transaction.index += 1
audible_transaction

,timestamp,user_id,book_id,country
1,2021-05-01 00:00:01,ad8eca41,1584,Portugal
2,2021-05-01 00:00:03,561b26c1,829,United States of America
3,2021-05-01 00:00:04,81f149e5,1391,Japan
4,2021-05-01 00:00:07,4f218413,1586,Taiwan
5,2021-05-01 00:00:18,a4066781,300,United States of America
...,...,...,...,...
1998817,2021-07-31 23:59:47,72da1411,600,Portugal
1998818,2021-07-31 23:59:48,620c27c8,462,Montenegro
1998819,2021-07-31 23:59:55,84fe88ab,1656,United States of America
1998820,2021-07-31 23:59:57,fd6cc4fc,522,United States of America


In [14]:
transaction = audible_transaction.merge(audible_data, how='left', left_on='book_id', right_on='Book_ID')
transaction

,timestamp,user_id,book_id,country,Book Title,Book Subtitle,Book Author,Book Narrator,Audio Runtime,Audiobook_Type,Categories,Rating,Total No. of Ratings,Price
0,2021-05-01 00:00:01,ad8eca41,1584,Portugal,Sycamore Row,None,John Grisham,Michael Beck,20 hrs and 46 mins,Unabridged Audiobook,"Mystery, Thriller & Suspense",4.6,25863.0,$35.00
1,2021-05-01 00:00:03,561b26c1,829,United States of America,The 5 Love Languages of Children,The Secret to Loving Children Effectively,Gary Chapman,Chris Fabry,5 hrs and 48 mins,Unabridged Audiobook,"Relationships, Parenting & Personal Development",4.8,2441.0,$19.59
2,2021-05-01 00:00:04,81f149e5,1391,Japan,The Guardians,A Novel,John Grisham,Michael Beck,11 hrs and 50 mins,Unabridged Audiobook,"Mystery, Thriller & Suspense",4.6,26781.0,$31.50
3,2021-05-01 00:00:07,4f218413,1586,Taiwan,How to Be a 3% Man,None,Corey Wayne,Corey Wayne,7 hrs and 33 mins,Unabridged Audiobook,"Relationships, Parenting & Personal Development",4.8,11459.0,$19.95
4,2021-05-01 00:00:18,a4066781,300,United States of America,Napoleon Hill's Outwitting the Devil,The Secret to Freedom and Success,Napoleon Hill,Dan John Miller,5 hrs and 51 mins,Unabridged Audiobook,Business & Careers,4.8,9305.0,$17.49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1998816,2021-07-31 23:59:47,72da1411,600,Portugal,The Time of Contempt,"The Witcher, Book 2",Andrzej Sapkowski,Peter Kenny,11 hrs and 55 mins,Unabridged Audiobook,Science Fiction & Fantasy,4.7,14344.0,$25.08
1998817,2021-07-31 23:59:48,620c27c8,462,Montenegro,Endurance,Shackleton's Incredible Voyage,Alfred Lansing,Simon Prebble,10 hrs and 20 mins,Unabridged Audiobook,History,4.7,18764.0,$24.95
1998818,2021-07-31 23:59:55,84fe88ab,1656,United States of America,The Spy and the Traitor,The Greatest Espionage Story of the Cold War,Ben Macintyre,John Lee,13 hrs and 20 mins,Unabridged Audiobook,Biographies & Memoirs,4.8,7150.0,$31.95
1998819,2021-07-31 23:59:57,fd6cc4fc,522,United States of America,Lilac Girls,A Novel,Martha Hall Kelly,Cassandra Campbell,17 hrs and 30 mins,Unabridged Audiobook,Literature & Fiction,4.7,19401.0,$31.50
